In [ ]:
from google.colab import drive

In [ ]:
# Importando as bibliotecas necessárias.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score , f1_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lendo os dados da tabela
df = pd.read_csv('/content/drive/MyDrive/Bases de dados - TCC/base_de_dados_tcc_tratada_v2.csv', low_memory=False)
df.head()

,League,Home,Away,INC,Round,Date,Temporada,Time,H_Score,A_Score,...,A_Offsides,H_Throw_in,A_Throw_in,H_Goalkeeper_Saves,A_Goalkeeper_Saves,H_Fouls,A_Fouls,H_Yellow_Cards,A_Yellow_Cards,Game Link
0,premier-league,Chelsea,Everton,"[""09' Goal_Home - Mata J."", ""14' Goal_Away - N...",38,5/19/2013,12-13,17:00,2,1,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,2.0,2.0,https://www.flashscore.com/match/CSiF8w91/#/ma...
1,premier-league,Liverpool,QPR,"[""08' Yellow_Home - Henderson J."", ""23' Goal_H...",38,5/19/2013,12-13,17:00,1,0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,3.0,https://www.flashscore.com/match/jJjJ7cO7/#/ma...
2,premier-league,Manchester City,Norwich,"[""26' Goal_Away - Pilkington A."", ""29' Goal_Ho...",38,5/19/2013,12-13,17:00,2,3,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.0,https://www.flashscore.com/match/Szz2Wt18/#/ma...
3,premier-league,Newcastle,Arsenal,"[""23' Yellow_Home - Gutierrez J."", ""39' Yellow...",38,5/19/2013,12-13,17:00,0,1,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,2.0,https://www.flashscore.com/match/6RY5V0GE/#/ma...
4,premier-league,Southampton,Stoke,"[""05' Yellow_Away - Whitehead D."", ""28' Yellow...",38,5/19/2013,12-13,17:00,1,1,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,2.0,https://www.flashscore.com/match/bNdZDYps/#/ma...


In [ ]:
df = df[df['League'] == 'ligue-1']

# Display the filtered DataFrame
len(df)

3707

In [ ]:
# Criando a coluna com o identificador do jogo (ID_PARTIDA)

df = pd.DataFrame(df)
df = df.reset_index()
df = df.rename(columns = {"index": "ID_PARTIDA"})
df["ID_PARTIDA"] = df.index + 1

In [ ]:
# Alterando os resultados de string (Draw = empate, Home = vitória do time da casa, Away = vitória do time visitante), para
# inteiro (0 = empate, 1 = vitória do time da casa, 2 = vitória do time visitante)
for i in range(0,len(df)):
    if df["WIN"][i] == "Draw":
        df.at[i,"WIN"] = '0'
    if df["WIN"][i] == "Home":
        df.at[i,"WIN"] = '1'
    if df["WIN"][i] == "Away":
        df.at[i,"WIN"] = '2'

df["WIN"] = df["WIN"].astype(str).astype(int)


Total de jogos:  3707
Total de colunas:  58
Total de jogos ganhos em casa:  1644
Total de jogos ganhos pelo visitante:  1084
Total de jogos empatados:  979
Percentual de jogos ganhos em casa: 44.35%


In [ ]:
# Capturando a quantidade de partidas e colunas da base
matches = df.shape[0]
features = df.shape[1]

# Separando as vitórias do time da casa, visitante e empate
home_win = len(df[df.WIN==1])
away_win = len(df[df.WIN==2])
draw = len(df[df.WIN==0])
val = [home_win, draw, away_win]

# Percentual de vitórias do time da casa
win_rate = (float(home_win)/(matches)) *100

# Mostrando na tela os resultados em texto e gráfico
print ('Total de jogos: ', matches)
print ('Total de colunas: ', features)
print ('Total de jogos ganhos em casa: ', home_win)
print ('Total de jogos ganhos pelo visitante: ', away_win)
print ('Total de jogos empatados: ', draw)
print ('Percentual de jogos ganhos em casa: {:.2f}%'.format( win_rate ))
x = np.arange(3)
plt.bar(x, val)
plt.xticks(x, ('Home', 'Draw', 'Away'))
plt.show()

In [ ]:
features = df[['H_BET', 'A_BET', 'X_BET']]
labels = df['WIN']


print('Features')
print(features.head())
print('=====================')
print('Labels')
print(labels.head())

scaler = MinMaxScaler().fit(features)
features_scale = scaler.transform(features)

print('Features: ',features_scale.shape)
print(features_scale)

# Separação de treino e teste
# Treino [:2965]     representa 80% dos dados
# Teste [2965:3335]      representa 10% dos dados
# Previsão [2965:3335]     representa 10% dos dados

X_train = features_scale[:2965]
X_test = features_scale[2965:3335]
y_train = labels[:2965]
y_test = labels[2965:3335]

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

Features
   H_BET  A_BET  X_BET
0   3.22   2.04   3.20
1   3.02   2.75   2.40
2   1.85   3.87   3.20
3   2.40   2.65   3.15
4   1.92   3.60   3.20
Labels
0    2
1    0
2    1
3    2
4    0
Name: WIN, dtype: int64
Features:  (3707, 3)
[[0.09098497 0.01983365 0.05579399]
 [0.08263773 0.03497547 0.01764425]
 [0.03380634 0.05886116 0.05579399]
 ...
 [0.32804674 0.00575816 0.15593705]
 [0.01419032 0.15547025 0.16547449]
 [0.01669449 0.1490723  0.13209347]]
2965 2965
370 370


In [ ]:
# Rodando o modelo de Regressão Logística em sua versão multinomial

clf_LR = LogisticRegression(multi_class = 'multinomial', max_iter = 2000)
clf_LR.fit(X_train, y_train)
pred = clf_LR.predict(X_test)

lr_acc = accuracy_score(y_test, pred)

print('Regressão Logística')
print('')
print('Acurácia: {:.2f}%'.format(lr_acc*100))

Regressão Logística

Acurácia: 49.73%
F1 Score: 49.73%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# Rodando o modelo de Árvore de Decisão

clf_DT = DecisionTreeClassifier(random_state=42)
clf_DT.fit(X_train, y_train)
pred = clf_DT.predict(X_test)

dt_acc = accuracy_score(y_test, pred)

print('Árvore de Decisão')
print('')
print('Acurácia: {:.2f}%'.format(dt_acc*100))

Árvore de Decisão

Acurácia: 37.57%
F1 Score: 36.00%


In [ ]:
# Rodando o modelo de Naive Bayes

clf_NB = GaussianNB()
clf_NB.fit(X_train, y_train)
pred = clf_NB.predict(X_test)

nb_acc = accuracy_score(y_test, pred)

print('Naive Bayes')
print('')
print('Acurácia Naive Bayes: {:.2f}%'.format(nb_acc*100))

Naive Bayes

Acurácia Naive Bayes: 41.35%
F1 Score: 41.35%


In [ ]:
# Inicializando o classificador de Rede Neural para classificação multiclasse
clf_NN = MLPClassifier(hidden_layer_sizes=(7,), max_iter=2000, random_state=42, activation='relu', solver='adam')

# Treinando o modelo
clf_NN.fit(X_train, y_train)

# Fazendo previsões
pred_nn = clf_NN.predict(X_test)

# Calculando Acurácia
nn_acc = accuracy_score(y_test, pred_nn)

# Exibindo os resultados
print('Rede Neural')
print('')
print('Acurácia: {:.2f}%'.format(nn_acc * 100))

Rede Neural

Acurácia: 50.27%
F1 Score: 37.81%


In [ ]:
previsao = features_scale[3235:]

game_id_full = df['ID_PARTIDA']
game_id_prev = game_id_full[3235:]

res_full = df['WIN']
res_prev = res_full[3235:]

pred = clf_NN.predict(previsao)

df = pd.DataFrame({'Real': res_prev, 'Previsão': pred, 'ID_PARTIDA': game_id_prev})

print(df)

      Real  Previsão  ID_PARTIDA
3235     2         1        3236
3236     0         1        3237
3237     2         2        3238
3238     1         2        3239
3239     1         1        3240
...    ...       ...         ...
3702     2         1        3703
3703     0         1        3704
3704     2         2        3705
3705     0         1        3706
3706     0         1        3707

[472 rows x 3 columns]


<Axes: xlabel='Previsão', ylabel='Real'>

In [ ]:
#confusion Matrix

df = pd.DataFrame(df,columns = ['Real','Previsão'])

cf_matrix = pd.crosstab(df['Real'], df['Previsão'], rownames = ['Real'] , colnames = ['Previsão'])

sns.heatmap(cf_matrix, annot = True, cmap = 'Blues', fmt = 'g')